# Use what you learned to fit a model to real data

Astronomers can measure the mass (Mp) and radius (Rp) of exoplanets, and have done so for more than 2000 out of ~5000 exoplanets.

These experimental values are then compared to theoretical predictions to gain information on the planet's composition.

However, these theoretical predictions rely on complex models that take a long time to simulate (a few minutes for every point!). For this reason, only a few points are generated in advance, and saved in a file.

Is there a way to obtain intermediate values without running the full simulation each time?

In this notebook, we will compare interpolation with fitting the simulation with a simple model.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from scipy import interpolate
from astropy.table import Table
%matplotlib inline

### Load data and create a linear interpolator

Don't hesitate to read the header of files (first few lines), they often contain useful information about the file's contents. The exoplanet catalog was downloaded from the <a href="https://exoplanetarchive.ipac.caltech.edu/">NASA exoplanet archive</a>. The theoretical simulations were performed using an interior structure model that is described here: <a href="https://exoplanets5.org/wp-content/uploads/1106.pdf">interior model of partially molten rocky planets based on PREM</a>.


In [ ]:
#load exoplanet data from file and unpack into variables
#genfromtxt has the additional option "filling_values" that np.loadtxt does not have
exoplanet_data = np.genfromtxt("catalog_exoplanets.dat",delimiter="\t",unpack=True,usecols=(1,2,3,4,5,6),filling_values=0.0)
rp,rp_err_up,rp_err_down,mp,mp_err_up,mp_err_down = exoplanet_data
print(f"Number of exoplanets: {len(rp)}") # look up the number of exoplanets by printing the number of values

#load the theoretical model that predicts the planet radius of Earth-like planets
theoretical_model = np.loadtxt("elike-band.dat",unpack=True,usecols=(0,1),skiprows=1)
mp_theory,rp_theory = theoretical_model
print(f"Number of simulations: {len(mp_theory)}") # number of simulated points

### Linear interpolation

In [ ]:
#create a function that, for any value of mass, computes the radius by linear interpolation
#this creates a function with the name "rad_interpol" that will take 1 argument (mass) and return 1 value (radius)
rad_interpol = interpolate.interp1d(mp_theory, rp_theory,kind="linear",fill_value="extrapolate")

#print out all known points as a table
sim_table = Table()
sim_table["Mp simulation"] = mp_theory
sim_table["Rp simulation"] = rp_theory
sim_table["Mp simulation"].format = "{:.5f}"
sim_table["Rp simulation"].format = "{:.5f}"
print(sim_table)

#test the interpolator for a value not in the table
mass_test = 1.0
print(f'mass = {mass_test:.5f}, radius interpolated = {rad_interpol(mass_test):.5f}')

In [ ]:
#Plot the exoplanet catalog, the theoretical model, and the interpolated values
f = plt.figure(figsize=(9,6))
plt.errorbar(mp,rp,xerr=(-mp_err_down,mp_err_up),yerr=(-rp_err_down,rp_err_up),
             marker='o',
             ls='None',
             alpha=0.15,
             color='black',
             markersize=5.0,
             label='Exoplanets')

plt.plot(mp_theory,rp_theory,
         marker='o',
         ls='None',
         alpha=1,
         color='blue',
         markersize=7.0,
         label='Earth-like composition (simulated)')

x_interp = np.linspace(np.min(mp_theory),np.max(mp_theory),500)
y_interp = rad_interpol(x_interp)
plt.plot(x_interp,y_interp,
         ls='--',
         alpha=1,
         color='blue',
         label='Earth-like composition (interpolated)')

plt.xlabel('Mass (Earth units)')
plt.ylabel('Radius (Earth units)')

plt.xlim(-1,25)
plt.ylim(0.5,3)
plt.legend(loc='lower right')

plt.show()

### Can we try to fit the data with a line?

The simulated data points clearly don't follow a linear relation, but we can still try to see what happens. For simplicity, let's use numpy polyfit.

In [ ]:
# fit a polynomial of degree 1
best_poly1 = np.polyfit(mp_theory,rp_theory,deg=1)

#the polyval function takes the polynomial coefficients (best_poly1) and some x-coordinates (mp_theory),
#and returns the array of y-coordinates at these points
x_fit = np.linspace(0 , 30 , 500)
y_fit = np.polyval(best_poly1 , x_fit)

#plot the data and the fit
f = plt.figure(figsize=(9,4))
plt.errorbar(mp_theory,rp_theory,fmt='o',label="Data")
plt.plot(x_fit , y_fit , "r-",label="Fit")
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

A line really doesn't fit the data well. How about a 2nd order polynomial?

In [ ]:
# fit a polynomial of degree 2
best_poly2 = np.polyfit(mp_theory,rp_theory,deg=2)

#the polyval function takes the polynomial coefficients (best_poly1) and some x-coordinates (mp_theory),
#and returns the array of y-coordinates at these points
x_fit = np.linspace(0 , 30 , 500)
y_fit = np.polyval(best_poly2 , x_fit)

#plot the data and the fit
f = plt.figure(figsize=(9,4))
plt.errorbar(mp_theory,rp_theory,fmt='o',label="Data")
plt.plot(x_fit , y_fit , "r-",label="Fit")
plt.xlabel('x')
plt.ylabel('y')
plt.ylim(0,2.5)
plt.legend()
plt.show()

## Curve fit

Polynomials, no matter the degree, really don't work well for this situation. A better option is to try different "shapes" (classes) of function.

In [ ]:
#for this case, a simple power law will suffice: Rp = Mp^a
def radius_model(mass,a):
    return mass**a

#perform the fit with initial guess a=1
params, params_cov = optimize.curve_fit(radius_model,mp_theory,rp_theory,p0=[1])

#get uncertainty
param_err = np.sqrt(np.diag(params_cov)) #extract the diagonal, and apply square root

a_fit = params[0]
a_err = param_err[0]

#print results
print(f'a_fit = {a_fit:.5f} +- {a_err:.5f}')


#### Let's check how good is the fit

In [ ]:
#create a figure with two vertically aligned panels
fig,(ax1,ax2) = plt.subplots(2,1,figsize=(9,6))

#top panel is data, interpolated, and fit
x_fit = np.linspace(0.1*np.min(mp_theory),2*np.max(mp_theory),500)
y_fit = radius_model(x_fit,*params)
ax1.plot(mp_theory,rp_theory,'bo',label='Simulated Data')
ax1.plot(x_interp,y_interp,ls='--',color='blue',label='Interpolated')
ax1.plot(x_fit,y_fit,'r-',label='Fit')
ax1.set_xticklabels([]) #remove xticks labels
ax1.set_xlabel('') #remove x label
ax1.set_ylabel('Radius')
ax1.legend(loc='lower right')
ax1.set_xlim(-1,25.0)
ax1.set_ylim(0.2,2.5)
ax1.grid()

#bottom panel is residuals
#residuals are the difference between the data and the fit
#they represent the "distance" between the data and the fit
#smaller values mean the model matches the data well
residuals = rp_theory-radius_model(mp_theory,*params)
ax2.plot(mp_theory,residuals,'bo',label='Residuals of the fit')
ax2.set_xlabel('Mass')
ax2.set_ylabel('Residuals')
ax2.legend(loc='upper right')
ax2.set_xlim(-1,25.0)
#ax2.set_ylim(0.2,2.5)
ax2.grid()

plt.subplots_adjust(hspace=0.0)

plt.show()

Pretty good, but can we do better?

In [ ]:
#try a slightly more refined model: Rp = Mp^a * ( 1 + b*log10(Mp) )
#there is a physical reason for choosing this specific shape,
#but for now let's just focus on how additional parameters improve the fit
def radius_model2(mass,a,b):
    return mass**a * ( 1.0 + b * np.log10(mass) )

#perform the fit with initial guess a=1 and b=1
params2, params_cov2 = optimize.curve_fit(radius_model2,mp_theory,rp_theory,p0=[1,1])

#get uncertainty
param_err2 = np.sqrt(np.diag(params_cov2)) #extract the diagonal, and apply square root

#print results
print(f'fit parameters [a,b] = {params2}')

In [ ]:
#create a figure with two vertically aligned panels
fig,(ax1,ax2) = plt.subplots(2,1,figsize=(9,6))

#top panel is data, interpolated, and fit
x_fit = np.linspace(0.1*np.min(mp_theory),2*np.max(mp_theory),500)
y_fit1 = radius_model(x_fit,*params)
y_fit2 = radius_model2(x_fit,*params2)
ax1.plot(mp_theory,rp_theory,'bo',label='Simulated Data')
ax1.plot(x_interp,y_interp,ls='--',color='blue',label='Interpolated')
ax1.plot(x_fit,y_fit1,'r-',label='Fit 1')
ax1.plot(x_fit,y_fit2,'g-',label='Fit 2')
ax1.set_xticklabels([]) #remove xticks labels
ax1.set_xlabel('') #remove x label
ax1.set_ylabel('Radius')
ax1.legend(loc='lower right')
ax1.set_xlim(-1,25.0)
ax1.set_ylim(0.2,2.5)
ax1.grid()

#bottom panel is residuals
#residuals are the difference between the data and the fit
#they represent the "distance" between the data and the fit
#smaller values mean the model matches the data well
residuals2 = rp_theory-radius_model2(mp_theory,*params2)
ax2.plot(mp_theory,residuals,'ro',label='Residuals 1')
ax2.plot(mp_theory,residuals2,'go',label='Residuals 2')
ax2.set_xlabel('Mass')
ax2.set_ylabel('Residuals')
ax2.legend(loc='upper right')
ax2.set_xlim(-1,25.0)
#ax2.set_ylim(0.2,2.5)
ax2.grid()

plt.subplots_adjust(hspace=0.0)

plt.show()

In both cases, the residuals are close to 0 on average, but one fit is clearly better than the other. The proper quantity to measure here is the standard deviation of the residuals. As we can see below, 

In [ ]:
#print out all known points as a table
residual_prop_table = Table([["fit 1","fit 2"],
                             [np.mean(residuals), np.mean(residuals2)], 
                             [np.std(residuals), np.std(residuals2)]], 
                            names=('','mean', 'std'))
residual_prop_table["mean"].format = "{:.5f}"
residual_prop_table["std"].format = "{:.5f}"
print(residual_prop_table)
print()
print(f"The second fit is {np.std(residuals)/np.std(residuals2):.1f} times better!!!")

#### Back to the case of the Earth

Remember how the interpolator predicted that an object with 1 times the mass of the Earth is not exactly 1 times the radius of the Earth? An additional simulation was performed and the returned value was 0.99683.

In [ ]:
print(f"radius interp = {rad_interpol(1.0):.5f}")
print(f"radius fit 1  = {radius_model(1.0,*params):.5f}")
print(f"radius fit 2  = {radius_model2(1.0,*params2):.5f}")
print(f"radius real   = {0.99683:.5f}")